In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

# Download the file and make it accessible to Spark
spark.sparkContext.addFile(url)
file_path = SparkFiles.get("home_sales_revised.csv")

# Read the CSV file into a DataFrame
df = spark.read.csv(file_path, header=True, inferSchema=True)


In [ ]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("sales_data_view")

In [ ]:
# Get the data types of the columns.
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)



In [ ]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
average_prices_4bd = spark.sql("""
    SELECT YEAR(Date) AS Year, ROUND(AVG(Price), 2) AS AveragePrice
    FROM sales_data_view
    WHERE Bedrooms = 4
    GROUP BY Year
    ORDER BY Year
""")

# Display the results
average_prices_4bd.show()


+----+------------+
|Year|AveragePrice|
+----+------------+
|2019|    300263.7|
|2020|   298353.78|
|2021|   301819.44|
|2022|   296363.88|
+----+------------+



In [ ]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
average_prices_3bd = spark.sql("""
    SELECT YEAR(Date) AS Year, ROUND(AVG(Price), 2) AS AveragePrice
    FROM sales_data_view
    WHERE Bedrooms = 3 AND Bathrooms = 3
    GROUP BY Year
    ORDER BY Year
""")

# Display the results
average_prices_3bd.show()


+----+------------+
|Year|AveragePrice|
+----+------------+
|2019|   287287.82|
|2020|   294204.16|
|2021|   294211.46|
|2022|   292725.69|
+----+------------+



In [ ]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
average_prices_custom = spark.sql("""
    SELECT YEAR(Date) AS Year, ROUND(AVG(Price), 2) AS AveragePrice
    FROM sales_data_view
    WHERE Bedrooms = 3 AND Bathrooms = 3 AND Floors = 2 AND sqft_living >= 2000
    GROUP BY Year
    ORDER BY Year
""")

# Display the results
average_prices_custom.show()


+----+------------+
|Year|AveragePrice|
+----+------------+
|2019|   289859.14|
|2020|   292289.09|
|2021|   296330.96|
|2022|   290242.99|
+----+------------+



In [ ]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

# Perform the average price calculation using Spark SQL
result = spark.sql("""
    SELECT ROUND(AVG(view), 2) AS AverageViewRating
    FROM sales_data_view
    WHERE price >= 350000
""")

# Extract the result
average_view_rating = result.collect()[0][0]

print("Average View Rating:", average_view_rating)
print("--- %s seconds ---" % (time.time() - start_time))


Average View Rating: 32.26
--- 0.5610215663909912 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("sales_data_view")

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('sales_data_view')

True

In [ ]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

# Perform the query using Spark SQL on the cached data
result = spark.sql("""
    SELECT *
    FROM sales_data_view
    WHERE view <= 5 AND price >= 350000
""")

# Collect the result to trigger the execution
result.collect()

print("--- %s seconds ---" % (time.time() - start_time))

--- 1.2032544612884521 seconds ---


Uncached runtime was faster

In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").parquet("formatted_home_sales_partitioned.parquet")

In [ ]:
# 11. Read the parquet formatted data.
parquet_df = spark.read.parquet("formatted_home_sales_partitioned.parquet")


In [ ]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("parquet_data_view")


In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

# Perform the query using Spark SQL on the Parquet DataFrame
result = spark.sql("""
    SELECT ROUND(AVG(view), 2) AS AverageViewRating
    FROM parquet_data_view
    WHERE price >= 350000
""")

# Extract the result
average_view_rating = result.collect()[0][0]

print("Average View Rating:", average_view_rating)
print("--- %s seconds ---" % (time.time() - start_time))

Average View Rating: 32.26
--- 0.3864879608154297 seconds ---


Parquet formatted data is faster than cached version.

In [ ]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("sales_data_view")

In [ ]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached("sales_data_view")

if is_cached:
    print("The 'sales_data_view' temporary table is still cached.")
else:
    print("The 'sales_data_view' temporary table is not cached.")


The 'sales_data_view' temporary table is not cached.
